In [197]:
from pandasticsearch import Select
from elasticsearch import Elasticsearch
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()

In [198]:
#엘라스틱서치 연결
es = Elasticsearch(hosts='192.168.1.66', port=9206, timeout=30, max_retries=10, retry_on_timeout=True)

In [199]:
def make_body(start, size=1000):
    #쿼리 정의 (전체 데이터 불러오기) 
    body= {
        "track_total_hits" : True,
        "from": start,
        "size": start+size,
        "sort": [
  {
    "send_timestamp": {
      "order": "desc"
    }
  }
], 
        "query": {
    "bool":{
      "filter": [
        {
          "exists": {
            "field": "NamedEntity"
          }
        }, 
        {
          "range": {
            "send_timestamp": {
              "gte": "2021-08-11T00:00:00.000Z",
              "format": "strict_date_optional_time"
            }
          }
        }
        
        ,{"terms": {
          "YNewsML.Metadata.Class.ClassCode.code": [
            "606001",
            "606002",
            "606003",
            "606004",
            "606005",
            "606006",
            "606007",
            "606008",
            "606009",
            "606010",
            "606011",
            "606012",
            "606013",
            "607001",
            "607002",
            "607003",
            "607004",
            "607005",
            "607006"
          ]
        }}
        
      ]
  }}}
    return body

In [200]:
index = 'rens_test'
size = 1000
#Data 전체 수 가져오기
total = es.search(index= index, body=make_body(start=0))['hits']['total']['value']
#Data 형태 보기
total1 = es.search(index= index, body=make_body(start=0))

total

13474

In [201]:
size = 10000
index ='rens_test'
df = pd.DataFrame()
for i in tqdm(range(0, total+size, size)):
        result = es.search(index= index, sort='send_timestamp', body=make_body(start=i, size=size), request_timeout=30)
        result1 = Select.from_dict(result).to_pandas()
        df = pd.concat([df,result1])

100%|██████████| 3/3 [00:04<00:00,  1.59s/it]


In [202]:
df.columns

Index(['_index', '_type', '_id', '_score', 'YNewsML.Header.SendTime',
       'YNewsML.Header.Action', 'YNewsML.Header.ServiceType',
       'YNewsML.Header.ContentID', 'YNewsML.Header.SendDate',
       'YNewsML.Metadata.Category.code', 'YNewsML.Metadata.Category.name',
       'YNewsML.Metadata.Credit', 'YNewsML.Metadata.DeliCode',
       'YNewsML.Metadata.Region.RegionCode', 'YNewsML.Metadata.Href',
       'YNewsML.Metadata.Desk.code', 'YNewsML.Metadata.Desk.content',
       'YNewsML.Metadata.Class.ClassCode', 'YNewsML.Metadata.Urgency',
       'YNewsML.Metadata.Source', 'YNewsML.Metadata.Writer.uid',
       'YNewsML.Metadata.Writer.content', 'YNewsML.NewsContent.LangType',
       'YNewsML.NewsContent.Title', 'YNewsML.NewsContent.Body', 'knn_vector',
       'NamedEntity', 'Keyword', 'send_timestamp', 'Summarization',
       'Quotation', 'YNewsML.Metadata.Region.RegionCode.code',
       'YNewsML.Metadata.Region.RegionCode.name',
       'YNewsML.NewsContent.Comment', 'YNewsML.NewsContent.

In [203]:
df = df.reset_index().drop('index',axis=1)
df

,_index,_type,_id,_score,YNewsML.Header.SendTime,YNewsML.Header.Action,YNewsML.Header.ServiceType,YNewsML.Header.ContentID,YNewsML.Header.SendDate,YNewsML.Metadata.Category.code,...,YNewsML.Metadata.Attribute.AttributeCode.code,YNewsML.Metadata.Attribute.AttributeCode.name,YNewsML.Metadata.Class.ClassCode.code,YNewsML.Metadata.Class.ClassCode.name,YNewsML.RefContent.Title,YNewsML.RefContent.Href,YNewsML.RefContent.ContentID,YNewsML.RefContent.type,YNewsML.Metadata.Attribute.AttributeCode,YNewsML.Metadata.Embargo
0,rens_test,_doc,AKR20221231041400061,None,232059,C,AKR0,AKR20221231041400061,20221231,07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rens_test,_doc,AKR20221231035000099,None,180813,C,AKR0,AKR20221231035000099,20221231,06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rens_test,_doc,AKR20221231025151003,None,161620,U,AKR0,AKR20221231025151003,20221231,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rens_test,_doc,AKR20221231025100003,None,125734,C,AKR0,AKR20221231025100003,20221231,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rens_test,_doc,AKR20221231022000076,None,115125,C,AKR0,AKR20221231022000076,20221231,06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13469,rens_test,_doc,AKR20220608057300003,None,103309,C,AKR0,AKR20220608057300003,20220608,02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13470,rens_test,_doc,AKR20220607103300530,None,100007,C,AKR0,AKR20220607103300530,20220608,03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13471,rens_test,_doc,PYH20220608015200017,None,092049,C,PYH0,PYH20220608015200017,20220608,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13472,rens_test,_doc,AKR20220607168000017,None,090002,C,AKR0,AKR20220607168000017,20220608,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
df[df['YNewsML.Header.SendDate'] == 20221030]['YNewsML.NewsContent.Title'].tolist()

['[이태원 참사] "외국인 사망26명·부상15명…해당공관에 위로서한"',
 '[이태원 참사] "핼러윈 휴업 부탁"…마포구, 홍대거리 계도(종합)',
 '[이태원 참사] 위험 조짐 있었는데…막을 순 없었나(종합)',
 '[이태원 참사] 대검 "이태원 참사 희생자 122명 검시 완료"',
 '[이태원 참사] 경찰 "사망자 154명…153명 신원 확인"',
 "핼러윈의 비극…이태원 '압사 참사' 153명 사망(종합3보)",
 '[이태원 참사] 주한 일본대사관 "일본인 여성 2명 사망"',
 '[이태원 참사] 코리아세일페스타 개막행사 등 취소(종합)',
 '[이태원 참사] 중대본 "중상자 37명으로 늘어…사망 153명"(종합)',
 '[이태원 참사] "조금만 더 신경 썼어도"…아쉬운 경찰 대처',
 '[이태원 참사] "CPR 흔적도 없더라" 취업 상경 20대 딸, 고향 친구와 함께 참변',
 '[이태원 참사] "가장 역할한 딸"…떠나간 그리운 얼굴들(종합)',
 '통제로 인적 끊긴 이태원 거리',
 '통제로 인적 드문 이태원 거리',
 '통제로 인적 드문 이태원 거리',
 '통제로 인적 드문 이태원 거리',
 '통제로 인적 드문 이태원 거리',
 '계속되는 추모의 발길',
 '이태원 참사, 추모의 메시지',
 '이태원 참사, 추모의 메시지',
 '[괴산 지진] 재산피해 총 15건…인명피해 없어',
 '참사 하루, 계속되는 헌화와 추모',
 '참사 하루, 계속되는 헌화와 추모',
 '참사 하루, 계속되는 헌화와 추모',
 '이태원역 출구, 헌화와 추모의 공간',
 '이태원역 출구, 헌화와 추모의 공간',
 '[이태원 참사] 이란인 5명 사망 확인…이란대사관 "깊은 애도"',
 '[이태원 참사] 용산구, 연말까지 애도기간…"사고 수습 전력"(종합)',
 '[이태원 참사] 중상자 37명으로 늘어…사망 153명',
 '[이태원 참사] 남양주·구리시, 사망자 각 1명 확인',
 '[이태원 참사] 전북도 11월 5일까지 애도 기간 설정',
 '[이태원 참사] "이대로 못 보낸

In [215]:
df3 = df[['YNewsML.NewsContent.Title', 'YNewsML.NewsContent.Body',
       'send_timestamp']]

In [15]:
df3 = df[['_id','YNewsML.NewsContent.Title', 'YNewsML.NewsContent.Body',
       'NamedEntity', 'Keyword', 'send_timestamp', 'Summarization','YNewsML.Metadata.Class.ClassCode']]

In [16]:
df3 = df3.drop_duplicates(subset='YNewsML.NewsContent.Body').reset_index().drop('index',axis=1)

In [17]:
len(df3)

11057

In [218]:
# df3.to_csv('ES_test.csv',encoding='utf-8-sig',index=False)

In [219]:
# df3 = df3.to_json(force_ascii=False)

In [220]:
# df3 = df3[:10]

In [18]:
def cleansing(body, mode=None):
    if type(body) == np.ndarray:
        body = np.round(body.tolist(), 5)
        body = body.tolist()
        return body
    else:
        body = re.sub('<YNAOBJECT.*?/YNAOBJECT>', '', body, 0, re.I|re.S) # YNAOBJECT 태그 제거
        body = re.sub('<table.*?/table>', '', body, 0, re.I|re.S) # table 태그 제거
        
        if mode == 'meta':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[0]:]
            except AttributeError:
                pass
            return body
        
        if mode == 'sum':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[1]:]
            except AttributeError:
                pass
            
        
        body = re.sub(r'\([^)]*\)', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub(r'\[.*\]|\s-\s.*', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub(r'\<.*\>|\s-\s.*', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub('·', ' ', body, 0, re.I|re.S) # 변환
        body = re.sub('-', ' ', body, 0, re.I|re.S) # 변환
        body = re.sub(',', ' ', body, 0, re.I|re.S) # 변환
        body = re.sub('\(.*?연합뉴스\).*?=', '', body, 0, re.I|re.S) # 기자명 및 기자소속 제거
        body = re.sub('\(서울.*?연합뉴스\)', '', body, 0, re.I|re.S) # 기자소속 제거
        body = re.sub('yna.co.kr', '', body, 0, re.I|re.S) # 연합 url 제거
        body = re.sub('([a-zA-Z0-9\_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '', body, 0, re.I|re.S) # E-mail 제거
        body = re.sub('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', body, 0, re.I|re.S) # URL 제거
        body = re.sub('전문보기.*?\n', '', body, 0, re.I|re.S) # 전문보기 제거
        body = re.sub('&lt;.*?&gt;&gt;', '', body, 0, re.I|re.S) # 기타 안내문 제거
        body = re.sub('&lt;.*?&gt;', '', body, 0, re.I|re.S) # '&gt;'가 1개인 안내문 제거
        body = re.sub('&apos;', "'", body, 0, re.I|re.S) # &apos; ' 변환
        body = re.sub('&amp;', "&", body, 0, re.I|re.S) # &amp;; ' 변환        
        body = re.sub('-.*?_.*?\(끝\)', '', body, 0, re.I|re.S) # 9001887 속성에만 존재하는 불용어 제거
        body = re.sub('[#@*※|》■▲▼◀▶◇☎]-<>=+·↑', '', body, 0, re.I|re.S) # 특수문자 제거
        body = re.sub('…', '', body, 0, re.I|re.S) # … 제거
        body = re.sub('\(끝\)', '', body, 0, re.I|re.S) # 끝 태그 제거
        body = re.sub('[""]', '', body, 0, re.I|re.S) # "" 제거
#         body = re.sub(r'[^ A-Za-z가-힣+]',' ' , body, 0, re.I|re.S) # 한글, 알파벳을 제외한 나머지 제거


        paragraphs = body.split('\r\n') # 단락 분리
        if ' 기자' in paragraphs[-1]: del paragraphs[-1] # 끝 단락 기자명 제거
        body = []
        for paragraph in paragraphs:
            paragraph = re.sub('\[.+?\]', '', paragraph, 0, re.I|re.S)
            paragraph = ' '.join(paragraph.split()) # 문자열 중간 다중 공백 제거
            if len(paragraph) > 0:
                body.append(paragraph)
        
        if mode == 'quot':
            return body
        else:
            return ' '.join(body)

In [19]:
df3['YNewsML.NewsContent.Body'] = df3['YNewsML.NewsContent.Body'].apply(cleansing)

In [20]:
df3

,_id,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,NamedEntity,Keyword,send_timestamp,Summarization,YNewsML.Metadata.Class.ClassCode
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편 이영주 기자 = 31일 오후...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","[정자역, 신분당선, 선로, 고장, 구원열차, 탑승, 꺼졌, 1시간]",2022-12-31T23:20:59,[31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 ...,"[{'code': '0410005', 'name': '철도운수'}, {'code':..."
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,남아공 가스트럭 폭발사고 사망자 34명으로 늘어 김성진 특파원 = 남아프리카공화국에...,"[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","[남아공, 사망자, 폭발, 환자]",2022-12-31T18:08:13,"[남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사...","[{'code': 1101001, 'name': '국제일반'}, {'code': '..."
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)",선로에 떨어진 부직포 때문에SRT 운행 차질에 차량 고장까지 전날 선로 전기공급 차...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","[열차, 지연, GS건설, 고장, 전기공급, 차단, 통복터널]",2022-12-31T16:16:20,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':..."
3,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소 첫날 KTX SRT 최대...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","[열차, 지연, SR, 전기공급, 끊기, 통복터널]",2022-12-31T12:57:34,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':..."
4,AKR20221231022000076,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료,캄보디아 카지노 호텔 화재 사망자 최소 27명으로수색 종료 강종훈 특파원 = 지난 ...,"[{'code': 'LC', 'word': '캄보디아'}, {'code': 'QT'...","[화재, 27명, 태국인, 카지노, 뛰어내리, 훈센]",2022-12-31T11:51:25,[지난 28일 발생한 캄보디아 북서부 카지노 호텔 대형 화재로 인한 사망자가 최소 ...,"[{'code': 1101001, 'name': '국제일반'}, {'code': '..."
...,...,...,...,...,...,...,...,...
11052,AKR20220608057300003,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인",한국소비자원 편의점 업계와 여름철 먹거리 안전 관리 캠페인 이신영 기자 = 한국소비...,"[{'code': 'OG', 'word': '한국소비자원'}, {'code': 'D...","[냉동식품, gs, 가이드, 위생관리, 한국소비자원, 정례협의체]",2022-06-08T10:33:09,[한국소비자원은 여름철을 맞아 편의점 사업자 정례협의체와 함께 먹거리 안전을 위한 ...,"[{'code': '0201010', 'name': '소비자'}, {'code': ..."
11053,AKR20220607103300530,"소방청-가스안전공사, 신에너지 사고 대응 업무협약",소방청 가스안전공사 신에너지 사고 대응 업무협약 계승현 기자 = 소방청은 8일 한국...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","[한국가스안전공사, 사고수습, 수소경제, 기술교육, 소방청, 대원, 신에너지, 대응]",2022-06-08T10:00:07,[소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무...,"[{'code': '0606008', 'name': '안전관리'}, {'code':..."
11054,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호선...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lgu, 구축, 8호선, cctv, 스마트스테이션, 고도, 사진, 3d, 서울지하철]",2022-06-08T09:20:49,"[LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8...","[{'code': '0901010', 'name': '모바일'}, {'code': ..."
11055,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 완료 역사 내 사고에 신속 대응해...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lg유플러스하이트, 서울교통공사, 역무원, 8호선, 스마트스테이션, 보안장비]",2022-06-08T09:00:02,[LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업...,"[{'code': '0901010', 'name': '모바일'}, {'code': ..."


In [21]:
EV_word_all = []
for i in range(len(df3)):
    try:
        for j in range(len(df3['NamedEntity'][i])):
                EV_word_all.append([df3['_id'][i],df3['NamedEntity'][i][j]['code'],df3['NamedEntity'][i][j]['word']])
    except:
        EV_word_all.append([df3['_id'][i],np.nan])

In [22]:
pd.DataFrame(EV_word_all)

,0,1,2
0,AKR20221231041400061,AF,신분당선
1,AKR20221231041400061,LC,판교
2,AKR20221231041400061,LC,정자역
3,AKR20221231041400061,AF,전동차
4,AKR20221231041400061,TI,1시간가량
...,...,...,...
244204,AKR20220608017300530,DT,이날
244205,AKR20220608017300530,QT,2회
244206,AKR20220608017300530,QT,3대
244207,AKR20220608017300530,OG,지방고용노동관서


In [23]:
df3['YNewsML.NewsContent.Body'][1]

'남아공 가스트럭 폭발사고 사망자 34명으로 늘어 김성진 특파원 = 남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련 사망자가 34명으로 늘었다고 AFP 로이터통신 등이 30일 남아공 보건부 등을 인용해 보도했다. 보도에 따르면 가스 트럭 폭발사고 사망자는 당초 알려진 27명에서 이같이 증가했다. 사망자 34명 가운데 폭발 사고로 가장 피해가 큰 인근 탐보 메모리얼 병원 소속 보건직이 11명이고 사고 현장 주변 구경꾼 등 일반인이 23명이다. 사망자 중에는 9세 어린이도 포함됐다. 사고 당시 경제중심 요하네스버그 동쪽 복스버그를 지나던 가스 트럭은 6만L의 취사용 액화석유가스를 싣고 있었다. 트럭은 낮게 걸린 다리 밑을 지나려다가 끼어 마찰과 함께 불이 나면서 가스가 폭발했다. 현장에서 100ｍ가량 떨어진 병원 내 환자들과 직원 수십 명이 심한 화상을 입어 사망자가 늘어날 것으로 예상됐었다. 강력한 폭발로 인해 응급실 지붕이 붕괴하고 유리창들이 박살 났다. 정확한 사고 원인에 대한 조사는 아직 진행 중이며 시릴 라마포사 남아공 대통령은 유족들에게 위로를 전했다. sungjin@'

In [24]:
EV_word_all1 = []
for i in range(len(df3)):
    for j in range(len(df3['NamedEntity'][i])):
        if df3['NamedEntity'][i][j]['code'] == 'EV':
            EV_word_all1.append([df3['_id'][i],df3['NamedEntity'][i][j]['word']])

In [25]:
EV_word_all1 = pd.DataFrame(EV_word_all1).rename(columns={0:'ID',1:'EV'})

In [26]:
EV_word_all1

,ID,EV
0,AKR20221230139200004,이태원 참사
1,AKR20221230084800064,천년대종 타
2,AKR20221230109100530,제야의 종 타종
3,AKR20221230086300530,이태원 참사
4,AKR20221230086300530,태풍 힌남노
...,...,...
5075,PYH20220609131400013,풍수
5076,AKR20220609042700530,강진
5077,AKR20220608141000004,청계천 시민구조 합동모의훈련
5078,AKR20220608104500052,경남 소방드론 경진대회


In [27]:
def nospace(data):
    data = re.sub(' ','',data)
    return data

In [28]:
EV_word_all1['EV_nospace'] = EV_word_all1['EV'].apply(nospace)

In [29]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [30]:
EV_word_all1

,ID,EV,EV_nospace
0,AKR20221230139200004,이태원 참사,이태원참사
1,AKR20221230084800064,천년대종 타,천년대종타
2,AKR20221230109100530,제야의 종 타종,제야의종타종
3,AKR20221230086300530,이태원 참사,이태원참사
4,AKR20221230086300530,태풍 힌남노,태풍힌남노
...,...,...,...
5075,PYH20220609131400013,풍수,풍수
5076,AKR20220609042700530,강진,강진
5077,AKR20220608141000004,청계천 시민구조 합동모의훈련,청계천시민구조합동모의훈련
5078,AKR20220608104500052,경남 소방드론 경진대회,경남소방드론경진대회


In [31]:
list1 = pd.DataFrame(EV_word_all1[['EV_nospace']].value_counts()).reset_index()['EV_nospace']

In [32]:
list1

0           이태원참사
1              태풍
2           태풍힌남노
3         이태원압사참사
4           이태원사고
          ...    
1029        삼풍백화점
1030    삼풍백화점붕괴사고
1031      삼풍백화점사건
1032         삼풍사고
1033           兩會
Name: EV_nospace, Length: 1034, dtype: object

In [33]:
a = []
for i in range(len(list1)):
    for j in range(len(EV_word_all1)):
        if jaccard_similarity(list1[i],EV_word_all1['EV_nospace'][j]) > 0.45:
            a.append([jaccard_similarity(list1[i],EV_word_all1['EV'][j]),EV_word_all1['ID'][j],list1[i],EV_word_all1['EV'][j]])

In [34]:
b = pd.DataFrame(a)
b

,0,1,2,3
0,0.833333,AKR20221230139200004,이태원참사,이태원 참사
1,0.833333,AKR20221230086300530,이태원참사,이태원 참사
2,0.833333,AKR20221230091800004,이태원참사,이태원 참사
3,0.454545,AKR20221230091800004,이태원참사,10·29 이태원 참사
4,0.555556,AKR20221230091800004,이태원참사,이태원 참사 추모제
...,...,...,...,...
49938,0.500000,AKR20220627087800797,삼풍백화점사건,삼풍백화점 붕괴
49939,0.545455,AKR20220627087800797,삼풍백화점사건,삼풍백화점 붕괴 참사
49940,0.500000,AKR20221108075800502,삼풍사고,삼풍
49941,0.800000,AKR20221108075800502,삼풍사고,삼풍 사고


In [35]:
a

[[0.8333333333333334, 'AKR20221230139200004', '이태원참사', '이태원 참사'],
 [0.8333333333333334, 'AKR20221230086300530', '이태원참사', '이태원 참사'],
 [0.8333333333333334, 'AKR20221230091800004', '이태원참사', '이태원 참사'],
 [0.45454545454545453, 'AKR20221230091800004', '이태원참사', '10·29 이태원 참사'],
 [0.5555555555555556, 'AKR20221230091800004', '이태원참사', '이태원 참사 추모제'],
 [0.8333333333333334, 'AKR20221230046200530', '이태원참사', '이태원 참사'],
 [0.8333333333333334, 'AKR20221229150300004', '이태원참사', '이태원 참사'],
 [0.8333333333333334, 'AKR20221229084600055', '이태원참사', '이태원 참사'],
 [0.45454545454545453, 'AKR20221229084600055', '이태원참사', '10.29 이태원 참사'],
 [0.6, 'PYH20221229075200013', '이태원참사', '이태원'],
 [0.6, 'PYH20221229074800013', '이태원참사', '이태원'],
 [0.6, 'PYH20221229072600013', '이태원참사', '이태원'],
 [0.6, 'PYH20221229073400013', '이태원참사', '이태원'],
 [0.6, 'PYH20221229069100013', '이태원참사', '이태원'],
 [0.6, 'PYH20221229069500013', '이태원참사', '이태원'],
 [0.6, 'PYH20221229063600013', '이태원참사', '이태원'],
 [0.6, 'PYH20221229061500013', '이태원참사', '이태원'],
 [0.

In [36]:
b[2].unique().tolist()

['이태원참사',
 '태풍',
 '태풍힌남노',
 '이태원압사참사',
 '이태원사고',
 '힌남노',
 "태풍'힌남노'",
 "태풍'힌남노",
 '이태원',
 '재난대응안전한국훈련',
 '태풍난마돌',
 '10·29이태원참사',
 '난마돌',
 '세월호참사',
 '49재',
 '카타르월드컵',
 '매미',
 '수능',
 '안전한국훈련',
 '학동참사',
 '부산불꽃축제',
 '대테러종합훈련',
 'ASF',
 '월드컵',
 '하이선',
 '마이삭',
 '참사',
 '태풍난마돌이',
 '이태원참사희생자추모위령제',
 '10',
 '서울안전한마당',
 "태풍'난마돌'",
 '압사참사',
 '아프리카돼지열병',
 '코로나19',
 '사라',
 '재난대응긴급구조종합훈련',
 '이태원참사사고',
 '태풍매미',
 '우크라이나전쟁',
 '대학수학능력시험',
 '태풍무이파',
 '용산이태원참사',
 '을지연습훈련',
 '을지연습',
 "태풍'난마돌",
 '부산세계박람회',
 '집중호우',
 '우면산산사태',
 '산사태',
 '태풍마이삭',
 '불꽃축제',
 '10.29(이태원)참사희생자추모위령제',
 '지스타',
 '10·29참사',
 '세월호',
 '우루과이전',
 '태풍차바',
 '핼러윈축제',
 '학동붕괴참사',
 '조별리그',
 '수해',
 '이태원참사시민추모촛불:당신잘못이아닙니다',
 '한국전쟁',
 '지방선거',
 "태풍'송다'",
 '재난대응안전한국',
 '풍수',
 '차바',
 '대한민국',
 '허리케인이언',
 '5·18',
 '러시아월드컵',
 '지진',
 "태풍'송다",
 '루사',
 '포항지진',
 '세월호사고',
 '수능시험',
 '동일본대지진',
 '후쿠시마원전사고',
 '태풍루사',
 '인천국제공항',
 '여의도불꽃축제',
 '이태원핼러',
 '2차대전',
 '정기국회',
 '대선',
 '국민안전발명챌린지',
 '해맞이축제',
 '대한민국가스안전대상',
 '남북실무접촉',
 '화정아이파크붕괴사고',
 '안전문화대상',
 '태풍힌남노로',
 '이태

In [37]:
b.loc[b[2]=='대구변호사사무실방화사건'][3].unique()

array(['대구 지하철 방화 사건', '대구 방화참사', '대구 방화사건', '변호사사무실 방화사건',
       '대구 변호사사무실 방화사건', '변호사 사무실 방화사건', '대구 변호사사무소 방화사건',
       '변호사 사무실 방화 사건', '대구 화재 사건'], dtype=object)

In [38]:
a = pd.DataFrame(a).rename(columns={0:'jaccard'})

In [39]:
c = b[[2,3]].drop_duplicates().reset_index().drop('index',axis=1)
c

,2,3
0,이태원참사,이태원 참사
1,이태원참사,10·29 이태원 참사
2,이태원참사,이태원 참사 추모제
3,이태원참사,10.29 이태원 참사
4,이태원참사,이태원
...,...,...
2876,삼풍백화점사건,삼풍백화점 붕괴 참사
2877,삼풍백화점사건,삼풍백화점 사건
2878,삼풍사고,삼풍
2879,삼풍사고,삼풍 사고


In [40]:
c.to_csv('jaccard.csv',encoding='utf-8-sig',index=False)

In [41]:
c = pd.read_csv('jaccard.csv',encoding='utf-8-sig')

In [42]:
c[c['2'] == '대구변호사사무실방화사건']

,2,3
1251,대구변호사사무실방화사건,대구 지하철 방화 사건
1252,대구변호사사무실방화사건,대구 방화참사
1253,대구변호사사무실방화사건,대구 방화사건
1254,대구변호사사무실방화사건,변호사사무실 방화사건
1255,대구변호사사무실방화사건,대구 변호사사무실 방화사건
1256,대구변호사사무실방화사건,변호사 사무실 방화사건
1257,대구변호사사무실방화사건,대구 변호사사무소 방화사건
1258,대구변호사사무실방화사건,변호사 사무실 방화 사건
1259,대구변호사사무실방화사건,대구 화재 사건


In [43]:
c_1 = pd.DataFrame(c[['2']].value_counts()).reset_index()['2']

In [44]:
c_1

0       이태원압사참사
1         이태원참사
2       이태원참사사고
3        이태원참사의
4        이태원참사진
         ...   
1029        스마트
1030    스페이스넷8'
1031     스피드레이싱
1032         시민
1033         兩會
Name: 2, Length: 1034, dtype: object

In [45]:
d = []
for i in c_1:
    cond1 = c['2'] == i
    d.append([i,c.loc[cond1]['3'].tolist(),len(c.loc[cond1])])

In [46]:
d

[['이태원압사참사',
  ['이태원 참사',
   '10·29 이태원 참사',
   '이태원 참사 추모제',
   '10.29 이태원 참사',
   '이태원',
   '이태원참사진상',
   '이태원참사진',
   '용산 이태원 참사',
   '이태원참사',
   '10·29 이태원참사',
   '이태원 압사 참사',
   '이태원참사 사고',
   '이태원 참사 추모 위령제',
   '이태원 사고',
   '이태원 참사의',
   '이태원참사 추모법회',
   '이태원 참사 추모법회',
   '압사 참사',
   '이태원 참사 희생자 추모',
   '이태원 참사 청년 추모행',
   '이태원 압사 사고',
   '태원 참사',
   '이태원 참사 사고',
   '이태원 압사 참사 사건',
   '이태원사고',
   '이태원 압사참사'],
  26],
 ['이태원참사',
  ['이태원 참사',
   '10·29 이태원 참사',
   '이태원 참사 추모제',
   '10.29 이태원 참사',
   '이태원',
   '이태원참사진상',
   '이태원참사진',
   '용산 이태원 참사',
   '이태원참사',
   '10·29 이태원참사',
   '이태원 압사 참사',
   '이태원참사 사고',
   '이태원 참사 추모 위령제',
   '이태원 참사 희생자 49재',
   '이태원 사고',
   '이태원 참사의',
   '이태원참사 추모법회',
   '이태원 참사 추모법회',
   '이태원 참사 희생자 추모',
   '이태원 참사 청년 추모행',
   '이태원 압사 사고',
   '태원 참사',
   '이태원 참사 사고',
   '이태원 압사 참사 사건',
   '이태원사고',
   '이태원 압사참사'],
  26],
 ['이태원참사사고',
  ['이태원 참사',
   '10·29 이태원 참사',
   '이태원 참사 추모제',
   '10.29 이태원 참사',
   '이태원',
   '이태원참사진상',
   '이태원참사진',
   '용산 이태원 참사',
   '이

In [47]:
f = pd.DataFrame(d).rename(columns={0:'nospace',1:'word_list',2:'len'}).drop_duplicates(subset='word_list').reset_index().drop('index',axis=1)

In [48]:
f.to_csv('jaccard_result.csv',encoding='utf-8-sig',index=False)

In [49]:
f

,nospace,word_list,len
0,이태원압사참사,"[이태원 참사, 10·29 이태원 참사, 이태원 참사 추모제, 10.29 이태원 참...",26
1,이태원참사,"[이태원 참사, 10·29 이태원 참사, 이태원 참사 추모제, 10.29 이태원 참...",26
2,이태원참사사고,"[이태원 참사, 10·29 이태원 참사, 이태원 참사 추모제, 10.29 이태원 참...",25
3,이태원참사추모제,"[이태원 참사, 이태원 참사 추모제, 이태원참사진상, 이태원참사진, 용산 이태원 참...",20
4,이태원참사추모법회,"[이태원 참사, 이태원 참사 추모제, 이태원참사진상, 이태원참사진, 용산 이태원 참...",19
...,...,...,...
897,스마트,[스마트],1
898,스페이스넷8',[스페이스넷8'],1
899,스피드레이싱,[스피드레이싱],1
900,시민,[시민],1


In [50]:
for i in range(len(f['word_list'])):
    for j in range(len(f['word_list'])):
        try:
            common = len(set(f['word_list'][i]).intersection(f['word_list'][i+j]))
            if common > 1:
                try:
                    f['nospace'][i+j] = f['nospace'][i]
                except:
                    pass
        except:
            pass

<ipython-input-50-1467751057b0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['nospace'][i+j] = f['nospace'][i]


In [51]:
result = f.drop_duplicates(subset='nospace').reset_index().drop('index',axis=1)

In [52]:
result

,nospace,word_list,len
0,이태원압사참사,"[이태원 참사, 10·29 이태원 참사, 이태원 참사 추모제, 10.29 이태원 참...",26
1,대한민국,"[대한민국 범죄예방대, 대한민국 인터넷대상, 대한민국, 대한민국 인터넷 대상, 국민...",15
2,재난안전대응훈련,"[어린이 재난안전훈련, 재난대응 안전한국훈련, 재난 대응 안전한국훈련, 재난대응 안...",14
3,소방기술경연대회,"[소방경진대회, 소방기술경연대회, 전국 소방기술경연대회, 전국소방기술경연대회, 소방...",12
4,태풍,"[태풍, 태풍 노루, 태풍매미, 태풍 차바, 태풍 루사, 차바 태풍, 태풍 매미, ...",12
...,...,...,...
583,스마트,[스마트],1
584,스페이스넷8',[스페이스넷8'],1
585,스피드레이싱,[스피드레이싱],1
586,시민,[시민],1


In [53]:
word_list = []
for i in range(len(result)):
        if len(result['word_list'][i]) > 1:
            word_list.append(result['nospace'][i])

In [54]:
word_list

['이태원압사참사',
 '대한민국',
 '재난안전대응훈련',
 '소방기술경연대회',
 '태풍',
 '힌남노태풍',
 '대구변호사사무소방화사건',
 '축제',
 '학동참사',
 '제야의종타종행사',
 '세월호침몰사고',
 '태풍난마돌이',
 '월드컵',
 '2차전',
 '88올림픽',
 '전국장애인체육대회',
 '폭우산사태',
 '봉화광산매몰사고',
 '삼풍백화점사건',
 '고인돌전국마라톤대회',
 '에어리',
 '을지태극연습',
 '화재',
 '허리케인',
 '가을문화축제',
 '지진',
 '세계잼버리',
 '서해훼리',
 '풍수해',
 '산업안전보건강조',
 '부산엑스포',
 '마산국화축제',
 '안전골든벨',
 '스텔라데이지호사고',
 '울산문화축전',
 '화정아이파크',
 '조별리그',
 '해남미남축제',
 '거리응원전',
 '수능시험',
 '슈즈페스티벌',
 "트라세'",
 '막국수닭갈비축제',
 '중대재해예방산업안전포럼',
 '태풍무이파',
 '지방선거',
 '인형극',
 '재난안전콘퍼런스',
 '재난경감장관회의',
 '테러',
 '정기국회',
 '자살폭탄테러',
 '자동차안전협력회의',
 '창덕궁달빛기행',
 '우크라이나전쟁',
 '신당역살인사건',
 '스마트건설챌린지',
 '새울원전방사능방재합동훈련',
 '성수대교',
 '안전보건관리체계구축컨설팅중소기업우수사례발표대회',
 '양양',
 '산불',
 '안전문화포럼',
 '체르노빌',
 '추석농수특산물서울장터',
 '10',
 '국가재난관리역',
 'DXKOREA',
 '대황소상전국민족씨름경기',
 '해맞이축제',
 '프로야구',
 '항공',
 '핼러윈',
 '6.1강진',
 '런던협약',
 '가습기살균제사건',
 '한미연합훈련',
 '러브퍼레이드',
 '촛불집회',
 '촛불문화제',
 '카니발',
 '초량지하차도참사',
 'GSTAR)',
 '공무원한마음체육대회',
 '싸이흠뻑쇼',
 'UFL',
 '해운대엘시티계단오르기대회',
 '고흥유자석류축제',
 '한미정상회담',
 'COP26',
 'CES'

In [55]:
result_word = []
for i in range(len(result)):
    if result['nospace'][i] in (word_list):
            result_word.append(result.loc[i])

In [56]:
mapping_list = pd.DataFrame(result_word)

In [57]:
mapping_list

,nospace,word_list,len
0,이태원압사참사,"[이태원 참사, 10·29 이태원 참사, 이태원 참사 추모제, 10.29 이태원 참...",26
1,대한민국,"[대한민국 범죄예방대, 대한민국 인터넷대상, 대한민국, 대한민국 인터넷 대상, 국민...",15
2,재난안전대응훈련,"[어린이 재난안전훈련, 재난대응 안전한국훈련, 재난 대응 안전한국훈련, 재난대응 안...",14
3,소방기술경연대회,"[소방경진대회, 소방기술경연대회, 전국 소방기술경연대회, 전국소방기술경연대회, 소방...",12
4,태풍,"[태풍, 태풍 노루, 태풍매미, 태풍 차바, 태풍 루사, 차바 태풍, 태풍 매미, ...",12
...,...,...,...
151,태안유류사고,"[태안 유류사고, 태안 유류 사고]",2
152,긴급구조종합훈련,"[긴급구조 종합훈련, 재난대응 긴급구조 종합훈련]",2
153,산업재해노동자,"[산업재해 노동자, 산업재해 예방 유공자]",2
154,삼풍,"[삼풍, 삼풍 사고]",2


In [58]:
df3

,_id,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,NamedEntity,Keyword,send_timestamp,Summarization,YNewsML.Metadata.Class.ClassCode
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편 이영주 기자 = 31일 오후...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","[정자역, 신분당선, 선로, 고장, 구원열차, 탑승, 꺼졌, 1시간]",2022-12-31T23:20:59,[31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 ...,"[{'code': '0410005', 'name': '철도운수'}, {'code':..."
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,남아공 가스트럭 폭발사고 사망자 34명으로 늘어 김성진 특파원 = 남아프리카공화국에...,"[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","[남아공, 사망자, 폭발, 환자]",2022-12-31T18:08:13,"[남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사...","[{'code': 1101001, 'name': '국제일반'}, {'code': '..."
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)",선로에 떨어진 부직포 때문에SRT 운행 차질에 차량 고장까지 전날 선로 전기공급 차...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","[열차, 지연, GS건설, 고장, 전기공급, 차단, 통복터널]",2022-12-31T16:16:20,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':..."
3,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소 첫날 KTX SRT 최대...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","[열차, 지연, SR, 전기공급, 끊기, 통복터널]",2022-12-31T12:57:34,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':..."
4,AKR20221231022000076,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료,캄보디아 카지노 호텔 화재 사망자 최소 27명으로수색 종료 강종훈 특파원 = 지난 ...,"[{'code': 'LC', 'word': '캄보디아'}, {'code': 'QT'...","[화재, 27명, 태국인, 카지노, 뛰어내리, 훈센]",2022-12-31T11:51:25,[지난 28일 발생한 캄보디아 북서부 카지노 호텔 대형 화재로 인한 사망자가 최소 ...,"[{'code': 1101001, 'name': '국제일반'}, {'code': '..."
...,...,...,...,...,...,...,...,...
11052,AKR20220608057300003,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인",한국소비자원 편의점 업계와 여름철 먹거리 안전 관리 캠페인 이신영 기자 = 한국소비...,"[{'code': 'OG', 'word': '한국소비자원'}, {'code': 'D...","[냉동식품, gs, 가이드, 위생관리, 한국소비자원, 정례협의체]",2022-06-08T10:33:09,[한국소비자원은 여름철을 맞아 편의점 사업자 정례협의체와 함께 먹거리 안전을 위한 ...,"[{'code': '0201010', 'name': '소비자'}, {'code': ..."
11053,AKR20220607103300530,"소방청-가스안전공사, 신에너지 사고 대응 업무협약",소방청 가스안전공사 신에너지 사고 대응 업무협약 계승현 기자 = 소방청은 8일 한국...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","[한국가스안전공사, 사고수습, 수소경제, 기술교육, 소방청, 대원, 신에너지, 대응]",2022-06-08T10:00:07,[소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무...,"[{'code': '0606008', 'name': '안전관리'}, {'code':..."
11054,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호선...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lgu, 구축, 8호선, cctv, 스마트스테이션, 고도, 사진, 3d, 서울지하철]",2022-06-08T09:20:49,"[LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8...","[{'code': '0901010', 'name': '모바일'}, {'code': ..."
11055,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 완료 역사 내 사고에 신속 대응해...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lg유플러스하이트, 서울교통공사, 역무원, 8호선, 스마트스테이션, 보안장비]",2022-06-08T09:00:02,[LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업...,"[{'code': '0901010', 'name': '모바일'}, {'code': ..."


In [59]:
EV_word_all1 = []
for i in range(len(df3)):
    for j in range(len(df3['NamedEntity'][i])):
        if df3['NamedEntity'][i][j]['code'] == 'EV':
            EV_word_all1.append([df3['_id'][i],df3['NamedEntity'][i][j]['word']])

In [60]:
EV_word_all1 = pd.DataFrame(EV_word_all1).rename(columns={0:'PK',1:'EV'})

In [61]:
EV_word_all1

,PK,EV
0,AKR20221230139200004,이태원 참사
1,AKR20221230084800064,천년대종 타
2,AKR20221230109100530,제야의 종 타종
3,AKR20221230086300530,이태원 참사
4,AKR20221230086300530,태풍 힌남노
...,...,...
5075,PYH20220609131400013,풍수
5076,AKR20220609042700530,강진
5077,AKR20220608141000004,청계천 시민구조 합동모의훈련
5078,AKR20220608104500052,경남 소방드론 경진대회


In [62]:
df3['jaccard'] = np.nan

In [63]:
df3

,_id,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,NamedEntity,Keyword,send_timestamp,Summarization,YNewsML.Metadata.Class.ClassCode,jaccard
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편 이영주 기자 = 31일 오후...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","[정자역, 신분당선, 선로, 고장, 구원열차, 탑승, 꺼졌, 1시간]",2022-12-31T23:20:59,[31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 ...,"[{'code': '0410005', 'name': '철도운수'}, {'code':...",NaN
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,남아공 가스트럭 폭발사고 사망자 34명으로 늘어 김성진 특파원 = 남아프리카공화국에...,"[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","[남아공, 사망자, 폭발, 환자]",2022-12-31T18:08:13,"[남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사...","[{'code': 1101001, 'name': '국제일반'}, {'code': '...",NaN
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)",선로에 떨어진 부직포 때문에SRT 운행 차질에 차량 고장까지 전날 선로 전기공급 차...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","[열차, 지연, GS건설, 고장, 전기공급, 차단, 통복터널]",2022-12-31T16:16:20,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",NaN
3,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소 첫날 KTX SRT 최대...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","[열차, 지연, SR, 전기공급, 끊기, 통복터널]",2022-12-31T12:57:34,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",NaN
4,AKR20221231022000076,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료,캄보디아 카지노 호텔 화재 사망자 최소 27명으로수색 종료 강종훈 특파원 = 지난 ...,"[{'code': 'LC', 'word': '캄보디아'}, {'code': 'QT'...","[화재, 27명, 태국인, 카지노, 뛰어내리, 훈센]",2022-12-31T11:51:25,[지난 28일 발생한 캄보디아 북서부 카지노 호텔 대형 화재로 인한 사망자가 최소 ...,"[{'code': 1101001, 'name': '국제일반'}, {'code': '...",NaN
...,...,...,...,...,...,...,...,...,...
11052,AKR20220608057300003,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인",한국소비자원 편의점 업계와 여름철 먹거리 안전 관리 캠페인 이신영 기자 = 한국소비...,"[{'code': 'OG', 'word': '한국소비자원'}, {'code': 'D...","[냉동식품, gs, 가이드, 위생관리, 한국소비자원, 정례협의체]",2022-06-08T10:33:09,[한국소비자원은 여름철을 맞아 편의점 사업자 정례협의체와 함께 먹거리 안전을 위한 ...,"[{'code': '0201010', 'name': '소비자'}, {'code': ...",NaN
11053,AKR20220607103300530,"소방청-가스안전공사, 신에너지 사고 대응 업무협약",소방청 가스안전공사 신에너지 사고 대응 업무협약 계승현 기자 = 소방청은 8일 한국...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","[한국가스안전공사, 사고수습, 수소경제, 기술교육, 소방청, 대원, 신에너지, 대응]",2022-06-08T10:00:07,[소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무...,"[{'code': '0606008', 'name': '안전관리'}, {'code':...",NaN
11054,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호선...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lgu, 구축, 8호선, cctv, 스마트스테이션, 고도, 사진, 3d, 서울지하철]",2022-06-08T09:20:49,"[LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8...","[{'code': '0901010', 'name': '모바일'}, {'code': ...",NaN
11055,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 완료 역사 내 사고에 신속 대응해...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lg유플러스하이트, 서울교통공사, 역무원, 8호선, 스마트스테이션, 보안장비]",2022-06-08T09:00:02,[LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업...,"[{'code': '0901010', 'name': '모바일'}, {'code': ...",NaN


In [105]:
count = 0
for i in range(len(EV_word_all1)):
        for j in range(len(EV_word_all1)):
            if df3['_id'][i] == EV_word_all1['PK'][j]:
                    df3['jaccard'][i] = EV_word_all1['EV'][j]

<ipython-input-105-458dfae1da46>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['jaccard'][i] = EV_word_all1['EV'][j]


In [106]:
mapping_list['nospace'].tolist()

['이태원압사참사',
 '대한민국',
 '재난안전대응훈련',
 '소방기술경연대회',
 '태풍',
 '힌남노태풍',
 '대구변호사사무소방화사건',
 '축제',
 '학동참사',
 '제야의종타종행사',
 '세월호침몰사고',
 '태풍난마돌이',
 '월드컵',
 '2차전',
 '88올림픽',
 '전국장애인체육대회',
 '폭우산사태',
 '봉화광산매몰사고',
 '삼풍백화점사건',
 '고인돌전국마라톤대회',
 '에어리',
 '을지태극연습',
 '화재',
 '허리케인',
 '가을문화축제',
 '지진',
 '세계잼버리',
 '서해훼리',
 '풍수해',
 '산업안전보건강조',
 '부산엑스포',
 '마산국화축제',
 '안전골든벨',
 '스텔라데이지호사고',
 '울산문화축전',
 '화정아이파크',
 '조별리그',
 '해남미남축제',
 '거리응원전',
 '수능시험',
 '슈즈페스티벌',
 "트라세'",
 '막국수닭갈비축제',
 '중대재해예방산업안전포럼',
 '태풍무이파',
 '지방선거',
 '인형극',
 '재난안전콘퍼런스',
 '재난경감장관회의',
 '테러',
 '정기국회',
 '자살폭탄테러',
 '자동차안전협력회의',
 '창덕궁달빛기행',
 '우크라이나전쟁',
 '신당역살인사건',
 '스마트건설챌린지',
 '새울원전방사능방재합동훈련',
 '성수대교',
 '안전보건관리체계구축컨설팅중소기업우수사례발표대회',
 '양양',
 '산불',
 '안전문화포럼',
 '체르노빌',
 '추석농수특산물서울장터',
 '10',
 '국가재난관리역',
 'DXKOREA',
 '대황소상전국민족씨름경기',
 '해맞이축제',
 '프로야구',
 '항공',
 '핼러윈',
 '6.1강진',
 '런던협약',
 '가습기살균제사건',
 '한미연합훈련',
 '러브퍼레이드',
 '촛불집회',
 '촛불문화제',
 '카니발',
 '초량지하차도참사',
 'GSTAR)',
 '공무원한마음체육대회',
 '싸이흠뻑쇼',
 'UFL',
 '해운대엘시티계단오르기대회',
 '고흥유자석류축제',
 '한미정상회담',
 'COP26',
 'CES'

In [109]:
df4 = df3.fillna('none').reset_index().drop('index',axis=1)

In [110]:
df4

,_id,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,NamedEntity,Keyword,send_timestamp,Summarization,YNewsML.Metadata.Class.ClassCode,jaccard
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편 이영주 기자 = 31일 오후...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","[정자역, 신분당선, 선로, 고장, 구원열차, 탑승, 꺼졌, 1시간]",2022-12-31T23:20:59,[31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 ...,"[{'code': '0410005', 'name': '철도운수'}, {'code':...",none
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,남아공 가스트럭 폭발사고 사망자 34명으로 늘어 김성진 특파원 = 남아프리카공화국에...,"[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","[남아공, 사망자, 폭발, 환자]",2022-12-31T18:08:13,"[남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사...","[{'code': 1101001, 'name': '국제일반'}, {'code': '...",none
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)",선로에 떨어진 부직포 때문에SRT 운행 차질에 차량 고장까지 전날 선로 전기공급 차...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","[열차, 지연, GS건설, 고장, 전기공급, 차단, 통복터널]",2022-12-31T16:16:20,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none
3,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소 첫날 KTX SRT 최대...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","[열차, 지연, SR, 전기공급, 끊기, 통복터널]",2022-12-31T12:57:34,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none
4,AKR20221231022000076,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료,캄보디아 카지노 호텔 화재 사망자 최소 27명으로수색 종료 강종훈 특파원 = 지난 ...,"[{'code': 'LC', 'word': '캄보디아'}, {'code': 'QT'...","[화재, 27명, 태국인, 카지노, 뛰어내리, 훈센]",2022-12-31T11:51:25,[지난 28일 발생한 캄보디아 북서부 카지노 호텔 대형 화재로 인한 사망자가 최소 ...,"[{'code': 1101001, 'name': '국제일반'}, {'code': '...",none
...,...,...,...,...,...,...,...,...,...
11052,AKR20220608057300003,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인",한국소비자원 편의점 업계와 여름철 먹거리 안전 관리 캠페인 이신영 기자 = 한국소비...,"[{'code': 'OG', 'word': '한국소비자원'}, {'code': 'D...","[냉동식품, gs, 가이드, 위생관리, 한국소비자원, 정례협의체]",2022-06-08T10:33:09,[한국소비자원은 여름철을 맞아 편의점 사업자 정례협의체와 함께 먹거리 안전을 위한 ...,"[{'code': '0201010', 'name': '소비자'}, {'code': ...",none
11053,AKR20220607103300530,"소방청-가스안전공사, 신에너지 사고 대응 업무협약",소방청 가스안전공사 신에너지 사고 대응 업무협약 계승현 기자 = 소방청은 8일 한국...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","[한국가스안전공사, 사고수습, 수소경제, 기술교육, 소방청, 대원, 신에너지, 대응]",2022-06-08T10:00:07,[소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무...,"[{'code': '0606008', 'name': '안전관리'}, {'code':...",none
11054,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호선...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lgu, 구축, 8호선, cctv, 스마트스테이션, 고도, 사진, 3d, 서울지하철]",2022-06-08T09:20:49,"[LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8...","[{'code': '0901010', 'name': '모바일'}, {'code': ...",none
11055,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 완료 역사 내 사고에 신속 대응해...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lg유플러스하이트, 서울교통공사, 역무원, 8호선, 스마트스테이션, 보안장비]",2022-06-08T09:00:02,[LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업...,"[{'code': '0901010', 'name': '모바일'}, {'code': ...",none


In [111]:
for i in range(len(df4['jaccard'])):
        for j in range(len(mapping_list)):
            if df4['jaccard'][i] in mapping_list['word_list'][j]:
                df4['jaccard'][i] = mapping_list['nospace'][j]

In [112]:
df4.to_csv('jaccard.csv',encoding='utf-8-sig',index=False)

In [113]:
jaccard = pd.read_csv('jaccard.csv',encoding='utf-8-sig')

In [114]:
jaccard['jaccard'].value_counts()

none                  9214
이태원압사참사               1254
재난안전대응훈련                34
4.9                     25
힌남노태풍                   21
                      ... 
국민생활과학기술포럼               1
전국의용소방대 성과·비전 보고대회       1
토론                       1
Vigilant Storm           1
한국전쟁                     1
Name: jaccard, Length: 260, dtype: int64

In [115]:
df3['Keyword'].tolist()

[['정자역', '신분당선', '선로', '고장', '구원열차', '탑승', '꺼졌', '1시간'],
 ['남아공', '사망자', '폭발', '환자'],
 ['열차', '지연', 'GS건설', '고장', '전기공급', '차단', '통복터널'],
 ['열차', '지연', 'SR', '전기공급', '끊기', '통복터널'],
 ['화재', '27명', '태국인', '카지노', '뛰어내리', '훈센'],
 ['시신', '국과수', '화재', '경기남부경찰청'],
 ['통영관광개발공사', '모노레일', '탈선', '재운행', '고민', '2021년'],
 ['서울시민', '2023년', '狡兎三窟'],
 ['기온', '기상청', '건조', '산불', '31일', '강원영서', '수도', '비슷하'],
 ['광교신도시', '방음터널', '강화유리'],
 ['강남소방서', '대피', '1층', '이송', '31일', '85명'],
 ['청주예술의전당', '해맞이'],
 ['방음터널', '화재', '세종시'],
 ['방음터널', '화재취약', '폴리메타크릴산메틸', '서울시', '강화유리', '구조'],
 ['방음터널', '방염성능기준', '폴리메타크릴산메틸', '도로교통연구원', '취약', '소방설비', '재료'],
 ['안전점검', '행안부', '사각지대', '행사', '보신각', '당부', '해넘이', '둘러봤'],
 ['행안부', '정부서울청사', '점검회의', '안전대책', '장관', '이상민', '30일', '2022.12.30', '다중밀집행사'],
 ['소방청', '경비국장', '안전대책', '119대응국장', '점검회의', '정부서울청사', '30일', '다중밀집행사', '연말연시'],
 ['행안부',
  '경찰청',
  '경비국장',
  '안전대책',
  '정부서울청사',
  '119대응국장',
  '이상민',
  '점검회의',
  '30일',
  '다중밀집행사'],
 ['행안부', '정부서울청사', '안전대책', '장관', '점검회의', '이상민', '2

In [116]:
df4

,_id,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,NamedEntity,Keyword,send_timestamp,Summarization,YNewsML.Metadata.Class.ClassCode,jaccard
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편 이영주 기자 = 31일 오후...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","[정자역, 신분당선, 선로, 고장, 구원열차, 탑승, 꺼졌, 1시간]",2022-12-31T23:20:59,[31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 ...,"[{'code': '0410005', 'name': '철도운수'}, {'code':...",none
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,남아공 가스트럭 폭발사고 사망자 34명으로 늘어 김성진 특파원 = 남아프리카공화국에...,"[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","[남아공, 사망자, 폭발, 환자]",2022-12-31T18:08:13,"[남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사...","[{'code': 1101001, 'name': '국제일반'}, {'code': '...",none
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)",선로에 떨어진 부직포 때문에SRT 운행 차질에 차량 고장까지 전날 선로 전기공급 차...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","[열차, 지연, GS건설, 고장, 전기공급, 차단, 통복터널]",2022-12-31T16:16:20,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none
3,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소 첫날 KTX SRT 최대...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","[열차, 지연, SR, 전기공급, 끊기, 통복터널]",2022-12-31T12:57:34,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none
4,AKR20221231022000076,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료,캄보디아 카지노 호텔 화재 사망자 최소 27명으로수색 종료 강종훈 특파원 = 지난 ...,"[{'code': 'LC', 'word': '캄보디아'}, {'code': 'QT'...","[화재, 27명, 태국인, 카지노, 뛰어내리, 훈센]",2022-12-31T11:51:25,[지난 28일 발생한 캄보디아 북서부 카지노 호텔 대형 화재로 인한 사망자가 최소 ...,"[{'code': 1101001, 'name': '국제일반'}, {'code': '...",none
...,...,...,...,...,...,...,...,...,...
11052,AKR20220608057300003,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인",한국소비자원 편의점 업계와 여름철 먹거리 안전 관리 캠페인 이신영 기자 = 한국소비...,"[{'code': 'OG', 'word': '한국소비자원'}, {'code': 'D...","[냉동식품, gs, 가이드, 위생관리, 한국소비자원, 정례협의체]",2022-06-08T10:33:09,[한국소비자원은 여름철을 맞아 편의점 사업자 정례협의체와 함께 먹거리 안전을 위한 ...,"[{'code': '0201010', 'name': '소비자'}, {'code': ...",none
11053,AKR20220607103300530,"소방청-가스안전공사, 신에너지 사고 대응 업무협약",소방청 가스안전공사 신에너지 사고 대응 업무협약 계승현 기자 = 소방청은 8일 한국...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","[한국가스안전공사, 사고수습, 수소경제, 기술교육, 소방청, 대원, 신에너지, 대응]",2022-06-08T10:00:07,[소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무...,"[{'code': '0606008', 'name': '안전관리'}, {'code':...",none
11054,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호선...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lgu, 구축, 8호선, cctv, 스마트스테이션, 고도, 사진, 3d, 서울지하철]",2022-06-08T09:20:49,"[LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8...","[{'code': '0901010', 'name': '모바일'}, {'code': ...",none
11055,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 완료 역사 내 사고에 신속 대응해...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lg유플러스하이트, 서울교통공사, 역무원, 8호선, 스마트스테이션, 보안장비]",2022-06-08T09:00:02,[LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업...,"[{'code': '0901010', 'name': '모바일'}, {'code': ...",none


In [177]:
df4['keyword_mapping'] = np.nan

In [137]:
mapping_list['word_list']

0      [이태원 참사, 10·29 이태원 참사, 이태원 참사 추모제, 10.29 이태원 참...
1      [대한민국 범죄예방대, 대한민국 인터넷대상, 대한민국, 대한민국 인터넷 대상, 국민...
2      [어린이 재난안전훈련, 재난대응 안전한국훈련, 재난 대응 안전한국훈련, 재난대응 안...
3      [소방경진대회, 소방기술경연대회, 전국 소방기술경연대회, 전국소방기술경연대회, 소방...
4      [태풍, 태풍 노루, 태풍매미, 태풍 차바, 태풍 루사, 차바 태풍, 태풍 매미, ...
                             ...                        
151                                  [태안 유류사고, 태안 유류 사고]
152                          [긴급구조 종합훈련, 재난대응 긴급구조 종합훈련]
153                              [산업재해 노동자, 산업재해 예방 유공자]
154                                          [삼풍, 삼풍 사고]
155                              [5·18민주화운동, 5·18 민주화운동]
Name: word_list, Length: 156, dtype: object

In [138]:
df4['Keyword']

0                  [정자역, 신분당선, 선로, 고장, 구원열차, 탑승, 꺼졌, 1시간]
1                                      [남아공, 사망자, 폭발, 환자]
2                      [열차, 지연, GS건설, 고장, 전기공급, 차단, 통복터널]
3                            [열차, 지연, SR, 전기공급, 끊기, 통복터널]
4                           [화재, 27명, 태국인, 카지노, 뛰어내리, 훈센]
                               ...                       
11052                [냉동식품, gs, 가이드, 위생관리, 한국소비자원, 정례협의체]
11053     [한국가스안전공사, 사고수습, 수소경제, 기술교육, 소방청, 대원, 신에너지, 대응]
11054    [lgu, 구축, 8호선, cctv, 스마트스테이션, 고도, 사진, 3d, 서울지하철]
11055        [lg유플러스하이트, 서울교통공사, 역무원, 8호선, 스마트스테이션, 보안장비]
11056                       [산업재해, 작업, 지방고용노동관서, 휴업, 3년간]
Name: Keyword, Length: 11057, dtype: object

In [174]:
df4['Keyword'][0]

['정자역', '신분당선', '선로', '고장', '구원열차', '탑승', '꺼졌', '1시간']

In [175]:
mapping_list['word_list'][0]

['이태원 참사',
 '10·29 이태원 참사',
 '이태원 참사 추모제',
 '10.29 이태원 참사',
 '이태원',
 '이태원참사진상',
 '이태원참사진',
 '용산 이태원 참사',
 '이태원참사',
 '10·29 이태원참사',
 '이태원 압사 참사',
 '이태원참사 사고',
 '이태원 참사 추모 위령제',
 '이태원 사고',
 '이태원 참사의',
 '이태원참사 추모법회',
 '이태원 참사 추모법회',
 '압사 참사',
 '이태원 참사 희생자 추모',
 '이태원 참사 청년 추모행',
 '이태원 압사 사고',
 '태원 참사',
 '이태원 참사 사고',
 '이태원 압사 참사 사건',
 '이태원사고',
 '이태원 압사참사']

In [176]:
len(set(df4['Keyword'][0]).intersection(mapping_list['word_list'][0]))

0

In [178]:
for i in range(len(df4['Keyword'])):
        for j in range(len(mapping_list['word_list'])):
                common = len(set(df4['Keyword'][i]).intersection(mapping_list['word_list'][j]))
                if common > 0:
                    df4['keyword_mapping'][i] = mapping_list['nospace'][j]

<ipython-input-178-3a668d8af515>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['keyword_mapping'][i] = mapping_list['nospace'][j]


In [179]:
df4[['keyword_mapping','jaccard']]

,keyword_mapping,jaccard
0,NaN,none
1,NaN,none
2,NaN,none
3,NaN,none
4,화재,none
...,...,...
11052,NaN,none
11053,NaN,none
11054,NaN,none
11055,NaN,none


In [180]:
df4['keyword_mapping'].value_counts()

이태원압사참사     523
화재          363
학동참사        344
힌남노태풍       279
태풍          225
           ... 
세계잼버리         1
대테러종합훈련       1
DXKOREA       1
런던협약          1
가습기살균제사건      1
Name: keyword_mapping, Length: 68, dtype: int64

In [181]:
df4['jaccard'].value_counts()

none                  9214
이태원압사참사               1254
재난안전대응훈련                34
4.9                     25
힌남노태풍                   21
                      ... 
국민생활과학기술포럼               1
전국의용소방대 성과·비전 보고대회       1
토론                       1
Vigilant Storm           1
한국전쟁                     1
Name: jaccard, Length: 260, dtype: int64

In [182]:
df4['key+jaccard'] = np.nan

In [190]:
for i in range(len(df4)):
        if df4['keyword_mapping'].fillna('none')[i] == df4['jaccard'][i]:
                df4['key+jaccard'][i] = df4['jaccard'][i]
        else:
            if df4['jaccard'][i] != 'none' and df4['keyword_mapping'].fillna('none')[i] == 'none':
                    df4['key+jaccard'][i] = df4['jaccard'][i]
            elif df4['keyword_mapping'].fillna('none')[i] != 'none' and df4['jaccard'][i] == 'none':
                    df4['key+jaccard'][i] = df4['keyword_mapping'].fillna('none')[i]
            else:
                df4['key+jaccard'][i] = np.nan
                

In [191]:
df4

,_id,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,NamedEntity,Keyword,send_timestamp,Summarization,YNewsML.Metadata.Class.ClassCode,jaccard,keyword_mapping,key+jaccard
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편 이영주 기자 = 31일 오후...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","[정자역, 신분당선, 선로, 고장, 구원열차, 탑승, 꺼졌, 1시간]",2022-12-31T23:20:59,[31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 ...,"[{'code': '0410005', 'name': '철도운수'}, {'code':...",none,NaN,none
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,남아공 가스트럭 폭발사고 사망자 34명으로 늘어 김성진 특파원 = 남아프리카공화국에...,"[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","[남아공, 사망자, 폭발, 환자]",2022-12-31T18:08:13,"[남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사...","[{'code': 1101001, 'name': '국제일반'}, {'code': '...",none,NaN,none
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)",선로에 떨어진 부직포 때문에SRT 운행 차질에 차량 고장까지 전날 선로 전기공급 차...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","[열차, 지연, GS건설, 고장, 전기공급, 차단, 통복터널]",2022-12-31T16:16:20,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none,NaN,none
3,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소 첫날 KTX SRT 최대...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","[열차, 지연, SR, 전기공급, 끊기, 통복터널]",2022-12-31T12:57:34,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none,NaN,none
4,AKR20221231022000076,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료,캄보디아 카지노 호텔 화재 사망자 최소 27명으로수색 종료 강종훈 특파원 = 지난 ...,"[{'code': 'LC', 'word': '캄보디아'}, {'code': 'QT'...","[화재, 27명, 태국인, 카지노, 뛰어내리, 훈센]",2022-12-31T11:51:25,[지난 28일 발생한 캄보디아 북서부 카지노 호텔 대형 화재로 인한 사망자가 최소 ...,"[{'code': 1101001, 'name': '국제일반'}, {'code': '...",none,화재,화재
...,...,...,...,...,...,...,...,...,...,...,...
11052,AKR20220608057300003,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인",한국소비자원 편의점 업계와 여름철 먹거리 안전 관리 캠페인 이신영 기자 = 한국소비...,"[{'code': 'OG', 'word': '한국소비자원'}, {'code': 'D...","[냉동식품, gs, 가이드, 위생관리, 한국소비자원, 정례협의체]",2022-06-08T10:33:09,[한국소비자원은 여름철을 맞아 편의점 사업자 정례협의체와 함께 먹거리 안전을 위한 ...,"[{'code': '0201010', 'name': '소비자'}, {'code': ...",none,NaN,none
11053,AKR20220607103300530,"소방청-가스안전공사, 신에너지 사고 대응 업무협약",소방청 가스안전공사 신에너지 사고 대응 업무협약 계승현 기자 = 소방청은 8일 한국...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","[한국가스안전공사, 사고수습, 수소경제, 기술교육, 소방청, 대원, 신에너지, 대응]",2022-06-08T10:00:07,[소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무...,"[{'code': '0606008', 'name': '안전관리'}, {'code':...",none,NaN,none
11054,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호선...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lgu, 구축, 8호선, cctv, 스마트스테이션, 고도, 사진, 3d, 서울지하철]",2022-06-08T09:20:49,"[LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8...","[{'code': '0901010', 'name': '모바일'}, {'code': ...",none,NaN,none
11055,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 완료 역사 내 사고에 신속 대응해...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lg유플러스하이트, 서울교통공사, 역무원, 8호선, 스마트스테이션, 보안장비]",2022-06-08T09:00:02,[LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업...,"[{'code': '0901010', 'name': '모바일'}, {'code': ...",none,NaN,none


In [192]:
df5 = df4[['_id', 'YNewsML.NewsContent.Title', 'YNewsML.NewsContent.Body',
       'NamedEntity', 'Keyword', 'send_timestamp', 'Summarization',
       'YNewsML.Metadata.Class.ClassCode','key+jaccard']]
df5

,_id,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,NamedEntity,Keyword,send_timestamp,Summarization,YNewsML.Metadata.Class.ClassCode,key+jaccard
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편 이영주 기자 = 31일 오후...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","[정자역, 신분당선, 선로, 고장, 구원열차, 탑승, 꺼졌, 1시간]",2022-12-31T23:20:59,[31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 ...,"[{'code': '0410005', 'name': '철도운수'}, {'code':...",none
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,남아공 가스트럭 폭발사고 사망자 34명으로 늘어 김성진 특파원 = 남아프리카공화국에...,"[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","[남아공, 사망자, 폭발, 환자]",2022-12-31T18:08:13,"[남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사...","[{'code': 1101001, 'name': '국제일반'}, {'code': '...",none
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)",선로에 떨어진 부직포 때문에SRT 운행 차질에 차량 고장까지 전날 선로 전기공급 차...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","[열차, 지연, GS건설, 고장, 전기공급, 차단, 통복터널]",2022-12-31T16:16:20,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none
3,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소 첫날 KTX SRT 최대...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","[열차, 지연, SR, 전기공급, 끊기, 통복터널]",2022-12-31T12:57:34,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none
4,AKR20221231022000076,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료,캄보디아 카지노 호텔 화재 사망자 최소 27명으로수색 종료 강종훈 특파원 = 지난 ...,"[{'code': 'LC', 'word': '캄보디아'}, {'code': 'QT'...","[화재, 27명, 태국인, 카지노, 뛰어내리, 훈센]",2022-12-31T11:51:25,[지난 28일 발생한 캄보디아 북서부 카지노 호텔 대형 화재로 인한 사망자가 최소 ...,"[{'code': 1101001, 'name': '국제일반'}, {'code': '...",화재
...,...,...,...,...,...,...,...,...,...
11052,AKR20220608057300003,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인",한국소비자원 편의점 업계와 여름철 먹거리 안전 관리 캠페인 이신영 기자 = 한국소비...,"[{'code': 'OG', 'word': '한국소비자원'}, {'code': 'D...","[냉동식품, gs, 가이드, 위생관리, 한국소비자원, 정례협의체]",2022-06-08T10:33:09,[한국소비자원은 여름철을 맞아 편의점 사업자 정례협의체와 함께 먹거리 안전을 위한 ...,"[{'code': '0201010', 'name': '소비자'}, {'code': ...",none
11053,AKR20220607103300530,"소방청-가스안전공사, 신에너지 사고 대응 업무협약",소방청 가스안전공사 신에너지 사고 대응 업무협약 계승현 기자 = 소방청은 8일 한국...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","[한국가스안전공사, 사고수습, 수소경제, 기술교육, 소방청, 대원, 신에너지, 대응]",2022-06-08T10:00:07,[소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무...,"[{'code': '0606008', 'name': '안전관리'}, {'code':...",none
11054,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호선...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lgu, 구축, 8호선, cctv, 스마트스테이션, 고도, 사진, 3d, 서울지하철]",2022-06-08T09:20:49,"[LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8...","[{'code': '0901010', 'name': '모바일'}, {'code': ...",none
11055,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 완료 역사 내 사고에 신속 대응해...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lg유플러스하이트, 서울교통공사, 역무원, 8호선, 스마트스테이션, 보안장비]",2022-06-08T09:00:02,[LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업...,"[{'code': '0901010', 'name': '모바일'}, {'code': ...",none


In [193]:
df4[df4['_id'] == 'AKR20221230090400061']

,_id,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,NamedEntity,Keyword,send_timestamp,Summarization,YNewsML.Metadata.Class.ClassCode,jaccard,keyword_mapping,key+jaccard
9,AKR20221230090400061,"불탄 광교 방음터널, 2년여 만에 재설치…불연성 '강화유리'로",불탄 광교 방음터널 2년여 만에 재설치불연성 '강화유리'로 수원시 내년 1월 착공해...,"[{'code': 'AF', 'word': '광교 방음터널'}, {'code': '...","[광교신도시, 방음터널, 강화유리]",2022-12-31T09:00:05,[화재로 소실된 뒤 2년 넘게 방치되고 있는 광교신도시 하동IC 고가차도 방음터널을...,"[{'code': '0501001', 'name': '사회일반'}, {'code':...",none,NaN,none


In [194]:
df5['key+jaccard'].value_counts()

none             7212
이태원압사참사          1129
화재                352
힌남노태풍             296
태풍                235
                 ... 
전국체전                1
산업재해노동자             1
MDSAP               1
급성심장정지조사 심포지엄       1
가습기살균제사건            1
Name: key+jaccard, Length: 238, dtype: int64

In [195]:
df5.to_csv('Keword_cluster.csv', encoding='utf-8-sig',index=False)

In [196]:
df5

,_id,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,NamedEntity,Keyword,send_timestamp,Summarization,YNewsML.Metadata.Class.ClassCode,key+jaccard
0,AKR20221231041400061,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편,신분당선 판교∼정자역 사이서 전동차 멈춰1시간가량 불편 이영주 기자 = 31일 오후...,"[{'code': 'AF', 'word': '신분당선'}, {'code': 'LC'...","[정자역, 신분당선, 선로, 고장, 구원열차, 탑승, 꺼졌, 1시간]",2022-12-31T23:20:59,[31일 오후 9시 40분께 경기도 성남시 신분당선 판교역과 정자역 사이 선로에서 ...,"[{'code': '0410005', 'name': '철도운수'}, {'code':...",none
1,AKR20221231035000099,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,남아공 가스트럭 폭발사고 사망자 34명으로 늘어 김성진 특파원 = 남아프리카공화국에...,"[{'code': 'LC', 'word': '남아공'}, {'code': 'AF',...","[남아공, 사망자, 폭발, 환자]",2022-12-31T18:08:13,"[남아프리카공화국에서 지난 크리스마스이브에 발생한 가스 트럭 폭발 사고와 관련, 사...","[{'code': 1101001, 'name': '국제일반'}, {'code': '...",none
2,AKR20221231025151003,"선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)",선로에 떨어진 부직포 때문에SRT 운행 차질에 차량 고장까지 전날 선로 전기공급 차...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'AF',...","[열차, 지연, GS건설, 고장, 전기공급, 차단, 통복터널]",2022-12-31T16:16:20,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none
3,AKR20221231025100003,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소,선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소 첫날 KTX SRT 최대...,"[{'code': 'AF', 'word': 'SRT'}, {'code': 'DT',...","[열차, 지연, SR, 전기공급, 끊기, 통복터널]",2022-12-31T12:57:34,[선로 전기공급 차단 문제로 수서고속철도(SRT) 열차 운행이 이틀째 차질을 빚고 ...,"[{'code': '0201001', 'name': '경제일반'}, {'code':...",none
4,AKR20221231022000076,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료,캄보디아 카지노 호텔 화재 사망자 최소 27명으로수색 종료 강종훈 특파원 = 지난 ...,"[{'code': 'LC', 'word': '캄보디아'}, {'code': 'QT'...","[화재, 27명, 태국인, 카지노, 뛰어내리, 훈센]",2022-12-31T11:51:25,[지난 28일 발생한 캄보디아 북서부 카지노 호텔 대형 화재로 인한 사망자가 최소 ...,"[{'code': 1101001, 'name': '국제일반'}, {'code': '...",화재
...,...,...,...,...,...,...,...,...,...
11052,AKR20220608057300003,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인",한국소비자원 편의점 업계와 여름철 먹거리 안전 관리 캠페인 이신영 기자 = 한국소비...,"[{'code': 'OG', 'word': '한국소비자원'}, {'code': 'D...","[냉동식품, gs, 가이드, 위생관리, 한국소비자원, 정례협의체]",2022-06-08T10:33:09,[한국소비자원은 여름철을 맞아 편의점 사업자 정례협의체와 함께 먹거리 안전을 위한 ...,"[{'code': '0201010', 'name': '소비자'}, {'code': ...",none
11053,AKR20220607103300530,"소방청-가스안전공사, 신에너지 사고 대응 업무협약",소방청 가스안전공사 신에너지 사고 대응 업무협약 계승현 기자 = 소방청은 8일 한국...,"[{'code': 'OG', 'word': '소방청가스안전공사'}, {'code':...","[한국가스안전공사, 사고수습, 수소경제, 기술교육, 소방청, 대원, 신에너지, 대응]",2022-06-08T10:00:07,[소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업무...,"[{'code': '0606008', 'name': '안전관리'}, {'code':...",none
11054,PYH20220608015200017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8호선...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lgu, 구축, 8호선, cctv, 스마트스테이션, 고도, 사진, 3d, 서울지하철]",2022-06-08T09:20:49,"[LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 8...","[{'code': '0901010', 'name': '모바일'}, {'code': ...",none
11055,AKR20220607168000017,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료",LGU+ 서울지하철 8호선에 '스마트 역사' 구축 완료 역사 내 사고에 신속 대응해...,"[{'code': 'OG', 'word': 'LGU+'}, {'code': 'AF'...","[lg유플러스하이트, 서울교통공사, 역무원, 8호선, 스마트스테이션, 보안장비]",2022-06-08T09:00:02,[LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사업...,"[{'code': '0901010', 'name': '모바일'}, {'code': ...",none
